<a href="https://colab.research.google.com/github/enki059/myworks/blob/main/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import logging
from abc import ABC, ABCMeta, abstractmethod
from typing import List
from datetime import date


# --- ЛОГИРОВАНИЕ ---

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler("project_log.txt", encoding="utf-8"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


# --- ИСКЛЮЧЕНИЯ ---

class InvalidDateError(Exception):
    """исключение при некорректной дате проекта"""
    pass

class PermissionDeniedError(Exception):
    """исключение при отсутствии прав доступа"""
    pass

class ProjectNotFoundError(Exception):
    """исключение при отсутствии проекта в хранилище"""
    pass


# --- ДЕКОРАТОР ДОСТУПА ---

def check_permissions(required_role):
    """декоратор для проверки прав доступа"""
    def decorator(func):
        def wrapper(self, *args, **kwargs):
            user = getattr(self, 'current_user', None)
            if not user or user.role != required_role:
                raise PermissionDeniedError(f"Доступ запрещён. Требуется роль: {required_role}")
            return func(self, *args, **kwargs)
        return wrapper
    return decorator


# --- СОТРУДНИК ---

class Employee:
    """класс для создания сотрудников"""
    def __init__(self, name: str, role: str):
        self.name = name
        self.role = role


# --- КОМАНДА ---

class Team:
    """класс для создания команды проекта с участниками и лидером"""
    def __init__(self):
        self.__members: List[Employee] = []
        self.leader: Employee = Employee("empty", "empty")

    def add_member(self, employee: Employee):
        """добавляет сотрудника в команду"""
        self.__members.append(employee)

    def remove_member(self, employee: Employee):
        """удаляет сотрудника из команды"""
        self.__members.remove(employee)

    def get_members(self):
        """возвращает список сотрудников в команде (без лидера)"""
        return self.__members

    def get_team_size(self):
        """возвращает количество участников команды"""
        return len(self.__members)

    def set_leader(self, leader: Employee):
        """устанавливает лидера"""
        self.leader = leader


# --- МЕТАКЛАСС ---

class ProjectMeta(ABCMeta):
    registry = {}

    def __new__(mcs, name, bases, namespace):
        cls = super().__new__(mcs, name, bases, namespace)
        if name != "Project":
            key = name.replace("Project", "").lower()
            ProjectMeta.registry[key] = cls
        return cls


# --- ИНТЕРФЕЙСЫ ---

class Trackable(ABC):
    @abstractmethod
    def track_progress(self) -> str:
        pass

class Reportable(ABC):
    @abstractmethod
    def generate_report(self) -> str:
        pass


# --- МИКСИНЫ ---

class LoggingMixin:
    def log_action(self, action: str):
        name = getattr(self, "name", "<неизвестный проект>")
        logger.info(f"Проект {name} обновлён: {action}")

class NotificationMixin:
    def send_notification(self, message: str):
        logger.info(f"[Уведомление] {message}")


# --- АБСТРАКТНЫЙ ПРОЕКТ ---

class Project(ABC, metaclass=ProjectMeta):
    """абстрактный класс для проекта"""
    def __init__(self, name: str, start_date: date, end_date: date, budget: float, status: str, team: Team):
        if end_date < start_date:
            raise InvalidDateError()
        self.__name = name
        self.__start_date = start_date
        self.__end_date = end_date
        self.__budget = budget
        self.__status = status
        self.__team = team

    @abstractmethod
    def calculate_progress(self) -> float:
        """рассчитывает прогресс в зависимости от проекта"""
        pass

    def __str__(self) -> str:
        return f"Название проекта: {self.name}, Статус: {self.status}"

    def to_dict(self) -> dict:
        """делает из объекта класса словарь"""
        return {
            "type": self.__class__.__name__,
            "name": self.name,
            "start_date": self.start_date.isoformat(),
            "end_date": self.end_date.isoformat(),
            "budget": self.budget,
            "status": self.status,
            "team": {
                "leader": {
                    "name": self.team.leader.name,
                    "role": self.team.leader.role
                },
                "members": [
                    {"name": m.name, "role": m.role} for m in self.team.get_members()
                ]
            }
        }

    @property
    def name(self):
        """геттер имени"""
        return self.__name

    @property
    def start_date(self):
        """геттер даты начала"""
        return self.__start_date

    @property
    def end_date(self):
        """геттер дедлайна"""
        return self.__end_date

    @property
    def budget(self):
        """геттер бюджета"""
        return self.__budget

    @property
    def status(self):
        """геттер статуса"""
        return self.__status

    @property
    def team(self):
        """геттер команды"""
        return self.__team

    @status.setter
    def status(self, new_status):
        """геттер статуса проекта"""
        self.__status = new_status

    def __eq__(self, other):
        return isinstance(other, Project) and self.budget == other.budget

    def __lt__(self, other):
        return isinstance(other, Project) and self.budget < other.budget

    def __gt__(self, other):
        return isinstance(other, Project) and self.budget > other.budget


# --- ПОДКЛАССЫ ПРОЕКТОВ ---

class SoftwareProject(Project, Trackable, Reportable, LoggingMixin, NotificationMixin):
    def __init__(self, name, start_date, end_date, budget, status, team, programming_languages):
        super().__init__(name, start_date, end_date, budget, status, team)
        self.__programming_languages = programming_languages

    def calculate_progress(self):
        """рассчитывает прогресс"""
        return 0

    def track_progress(self):
        """показывает прогресс"""
        return f"Прогресс по программному проекту: {self.calculate_progress()}% завершено"

    def generate_report(self):
        """генерирует отчет"""
        return f"Отчет по проекту {self.name}: использованные языки - {', '.join(self.programming_languages)}"

    def __str__(self):
        return f"Программный проект: {self.name}, языки: {', '.join(self.programming_languages)}"

    def to_dict(self):
        data = super().to_dict()
        data["programming_languages"] = self.programming_languages
        return data

    @property
    def programming_languages(self):
        return self.__programming_languages


class MarketingProject(Project, LoggingMixin, NotificationMixin):
    def __init__(self, name, start_date, end_date, budget, status, team, target_audience):
        super().__init__(name, start_date, end_date, budget, status, team)
        self.__target_audience = target_audience

    def calculate_progress(self):
        """рассчитывает прогресс"""
        return 1

    def track_progress(self):
        """показывает прогресс"""
        return f"Прогресс по маркетинговому проекту: {self.calculate_progress()}% завершено"

    def generate_report(self):
        """генерирует отчет"""
        return f"Отчет по маркетинговому проекту {self.name}: целевая аудитория — {self.__target_audience}"

    def __str__(self):
        return f"Маркетинговый проект: {self.name}, целевая аудитория: {self.__target_audience}"

    def to_dict(self):
        data = super().to_dict()
        data["target_audience"] = self.__target_audience
        return data

    @property
    def target_audience(self):
        """геттер для целевой аудитории"""
        return self.__target_audience


class ResearchProject(Project, LoggingMixin, NotificationMixin):
    def __init__(self, name, start_date, end_date, budget, status, team, research_field):
        super().__init__(name, start_date, end_date, budget, status, team)
        self.__research_field = research_field

    def calculate_progress(self):
        """рассчитывает прогресс"""
        return 2

    def track_progress(self):
        """показывает прогресс"""
        return f"Прогресс по маркетинговому проекту: {self.calculate_progress()}% завершено"

    def generate_report(self):
        """генерирует отчет"""
        return f"Отчет по исследовательскому проекту {self.name}: область исследования — {self.__research_field}"

    def __str__(self):
        return f"Исследовательский проект: {self.name}, область исследования: {self.__research_field}"

    def to_dict(self):
        data = super().to_dict()
        data["research_field"] = self.__research_field
        return data

    @property
    def research_field(self):
        """геттер для области исследования"""
        return self.__research_field


# --- ШАБЛОННЫЙ МЕТОД ---

class ProgressCalculator(ABC):
    def calculate_progress(self, project: Project) -> str:
        """шаблонный метод, рассчитывающий прогресс"""
        state = self.get_state(project)
        progress = self.compute_progress(state)
        return self.format_result(progress)

    @abstractmethod
    def get_state(self, project: Project):
        pass

    @abstractmethod
    def compute_progress(self, state):
        pass

    @abstractmethod
    def format_result(self, progress: float) -> str:
        pass


class SoftwareProgressCalculator(ProgressCalculator):
    def get_state(self, project: SoftwareProject):
        # считаем кол-во языков программирования как прогресс
        return len(project.programming_languages)

    def compute_progress(self, state):
        return min(state * 25, 100)  # >= 4 языка - 100%

    def format_result(self, progress: float) -> str:
        return f"Прогресс разработки: {progress:.1f}%"


class MarketingProgressCalculator(ProgressCalculator):
    def get_state(self, project: MarketingProject):
        return project.target_audience

    def compute_progress(self, state):
        return min(state / 1000 * 10, 100)  # масштабный охват - 100%

    def format_result(self, progress: float) -> str:
        return f"Прогресс маркетинга: {progress:.1f}%"


# --- ФАБРИКА ПРОЕКТОВ ---

class ProjectFactory:
    @staticmethod
    def create_project(project_type: str, **kwargs) -> Project:
        """создает проект"""
        cls = ProjectMeta.registry.get(project_type)
        if not cls:
            raise ValueError(f"Неизвестный тип проекта: {project_type}")
        return cls(**kwargs)

    @staticmethod
    def from_dict(data: dict) -> Project:
        project_type = data["type"].replace("Project", "").lower()
        cls = ProjectMeta.registry.get(project_type)
        if not cls:
            raise ProjectNotFoundError(f"Тип проекта '{project_type}' не зарегистрирован.")

        team = Team()
        leader_data = data["team"]["leader"]
        team.set_leader(Employee(leader_data["name"], leader_data["role"]))

        for member in data["team"]["members"]:
            team.add_member(Employee(member["name"], member["role"]))

        kwargs = {
            "name": data["name"],
            "start_date": date.fromisoformat(data["start_date"]),
            "end_date": date.fromisoformat(data["end_date"]),
            "budget": data["budget"],
            "status": data["status"],
            "team": team
        }

        if project_type == "software":
            kwargs["programming_languages"] = data.get("programming_languages", [])

        if project_type == "marketing":
            kwargs["target_audience"] = data.get("target_audience", [])

        if project_type == "research":
            kwargs["research_field"] = data.get("research_field", [])

        return cls(**kwargs)


# --- СЕРИАЛИЗАЦИЯ ---

def save_projects_to_file(projects: list[Project], filename: str):
    """сохраняет проекты в JSON-файл"""
    with open(filename, "w", encoding="utf-8") as f:
        json.dump([p.to_dict() for p in projects], f, indent=4, ensure_ascii=False)

def load_projects_from_file(filename: str) -> list[Project]:
    """загружает проекты из JSON-файла"""
    with open(filename, "r", encoding="utf-8") as f:
        data = json.load(f)
        return [ProjectFactory.from_dict(item) for item in data]


# --- ЦЕПОЧКА ОБЯЗАННОСТЕЙ ---

class BudgetHandler(ABC):
    def __init__(self):
        self._next_handler: BudgetHandler | None = None

    def set_next(self, handler: 'BudgetHandler') -> 'BudgetHandler':
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle_request(self, amount: float):
        pass


class ProjectLeader(BudgetHandler):
    def handle_request(self, amount: float):
        if amount <= 1000:
            print(f"Руководитель проекта одобрил изменение бюджета на {amount} у.е.")
        elif self._next_handler:
            print("Руководитель передаёт запрос дальше...")
            self._next_handler.handle_request(amount)
        else:
            print("Запрос отклонён: превышен лимит")


class FinanceDepartment(BudgetHandler):
    def handle_request(self, amount: float):
        if amount <= 5000:
            print(f"Финансовый отдел одобрил изменение бюджета на {amount} у.е.")
        elif self._next_handler:
            print("Финансовый отдел передаёт запрос дальше...")
            self._next_handler.handle_request(amount)
        else:
            print("Запрос отклонён: превышен лимит")


class Director(BudgetHandler):
    def handle_request(self, amount: float):
        print(f"Директор одобрил изменение бюджета на {amount} у.е.")


# --- ПРИМЕР ---

# создание команд
# программный проект
team = Team()
team.set_leader(Employee("Михаил", "manager"))
team.add_member(Employee("Иван", "developer"))
team.add_member(Employee("Ольга", "tester"))

# создание проекта с помощью фабрики
project_data = {
    "name": "Программный проект",
    "start_date": date(2025, 1, 1),
    "end_date": date(2025, 12, 31),
    "budget": 150000,
    "status": "В процессе",
    "team": team,
    "programming_languages": ["Python", "C++"]
}

project = ProjectFactory.create_project("software", **project_data)


# маркетинговый проект
team1 = Team()
team1.set_leader(Employee("Илья", "manager"))
team1.add_member(Employee("Софья", "developer"))
team1.add_member(Employee("Татьяна", "tester"))

# создание проекта с помощью фабрики
project_data1 = {
    "name": "Маркетинговый проект",
    "start_date": date(2025, 1, 1),
    "end_date": date(2025, 12, 31),
    "budget": 150000,
    "status": "В процессе",
    "team": team,
    "target_audience": "18-25 лет"
}

project1 = ProjectFactory.create_project("marketing", **project_data1)


# исследовательский проект
team2 = Team()
team2.set_leader(Employee("Эрнест", "mathematician"))
team2.add_member(Employee("Сергей", "researcher"))
team2.add_member(Employee("Олег", "researcher"))

# создание проекта с помощью фабрики
project_data2 = {
    "name": "Исследовательский проект",
    "start_date": date(2026, 8, 24),
    "end_date": date(2029, 12, 19),
    "budget": 0,
    "status": "В процессе",
    "team": team,
    "research_field": "Математика"
}

project2 = ProjectFactory.create_project("research", **project_data2)

# участники проекта
members = team.get_members()
print("Участники программного проекта:")
print(f"Лидер: {team.leader.name} ({team.leader.role})")
for member in members:
    print(f"- {member.name} ({member.role})")

members1 = team1.get_members()
print("Участники маркетингового проекта:")
print(f"Лидер: {team1.leader.name} ({team1.leader.role})")
for member in members1:
    print(f"- {member.name} ({member.role})")

members2 = team2.get_members()
print("Участники исследовательского проекта:")
print(f"Лидер: {team2.leader.name} ({team2.leader.role})")
for member in members2:
    print(f"- {member.name} ({member.role})")

# дальше происходят изменения только в программном проекте
# изменение статуса проекта
project.status = "Завершено"

# удаление участника
#team.remove_member(team.get_members()[1])  # Удаляем первого участника

# прогресс проекта
# progress = project.track_progress()
# print(f"Прогресс проекта: {progress}")

# генерация отчета
# report = project.generate_report()
# print(report)

print("\nШаблонный метод расчёта прогресса")

calc = SoftwareProgressCalculator()
print(calc.calculate_progress(project))

# логирование и отправка уведомлений
project.log_action("Изменен бюджет проекта")
project.send_notification("Проект требует внимания!")

# создание проекта с некорректной датой
try:
    invalid_project = ProjectFactory.create_project(
        "software",
        name="Неверный проект",
        start_date=date(2025, 5, 1),
        end_date=date(2024, 5, 1),
        budget=5000,
        status="В процессе",
        team=team,
        programming_languages=["Python"]
    )
except InvalidDateError:
    print("\nОшибка: Дата окончания не может быть раньше даты начала.")

# использование декоратора для проверки прав доступа
try:
    project.current_user = Employee("Олег", "developer")  # Пользователь с ролью "developer"
    project.log_action("Изменен статус проекта")  # Проверка прав доступа
except PermissionDeniedError as e:
    print(f"Ошибка прав доступа: {e}")

# сохранение проектов в файл
save_projects_to_file([project, project1, project2], "projects.json")

# загрузка проектов из файла
loaded_projects = load_projects_from_file("projects.json")
print("\nЗагруженные проекты:")
for p in loaded_projects:
    print(p)

Участники программного проекта:
Лидер: Михаил (manager)
- Иван (developer)
- Ольга (tester)
Участники маркетингового проекта:
Лидер: Илья (manager)
- Софья (developer)
- Татьяна (tester)
Участники исследовательского проекта:
Лидер: Эрнест (mathematician)
- Сергей (researcher)
- Олег (researcher)

Шаблонный метод расчёта прогресса
Прогресс разработки: 50.0%

Ошибка: Дата окончания не может быть раньше даты начала.

Загруженные проекты:
Программный проект: Программный проект, языки: Python, C++
Маркетинговый проект: Маркетинговый проект, целевая аудитория: 18-25 лет
Исследовательский проект: Исследовательский проект, область исследования: Математика
